In [1]:
d ={"key":"value","key1":"value2"}

In [6]:
from box import ConfigBox, config_box

In [3]:
d2 = ConfigBox({"key1":"a","key2":'b',"ke3":'x'}) 

In [8]:
d2 = ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}})

In [9]:
d2.artifacts_root

'artifacts'

In [106]:
def get_product (x:int, y:int) -> int:
    return x*y

In [107]:
get_product(4,5)

20

In [108]:
get_product(x=3,y="4")

'444'

In [109]:
#from ensure import ensure_annotations

In [110]:
%pwd

'd:\\MLproject\\First_project\\Project\\machine-learning-mlflow\\research'

In [111]:
from dataclasses import  dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path
    

In [112]:
from src.mlploject.utils.common import create_directories, read_yaml
from src.mlploject.constants import PARAMS_FILE_PATH,SCHEMA_FILE_PATH, CONFIG_FILE_PATH
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,#Path("config/config.yaml"),
                 params_filepath = PARAMS_FILE_PATH,#Path("params.yaml"),
                 schema_filepath =SCHEMA_FILE_PATH): #Path("schema.yaml")):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories(path_to_directories=[self.config.artifact_root])
        
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        
        return data_ingestion_config

In [113]:
import os
import urllib.request as request
import zipfile
from src.mlploject import logger
from src.mlploject.utils.common import get_size

In [114]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
            
            
    def extract_zip_file(self):
        """ extract zip file
        zipe file path: str
        Extract the zip file into the data directory
        function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [115]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'